# Biodiversity Intactness Index Change in Phoenix Subdivision

Author: Mariam Garcia

December 8, 2023

Github Repository Link : https://github.com/mariamkg00/eds-220-bii-phoenix/tree/main


## Dataset description

**First dataset** : The first dataset needed in order to analyze BII data will be accessed from the Microsoft Planetary Computer Catalog. We will *only* be looking at rasters 
from 2017 and 2020. This dataset estimates terrestrial Biodiversity Intactness as 100-meter gridded maps for the years 2017-2020.

**Second dataset** : The second dataset is a 2022 TIGER (Topologically Integrated Geographic Encoding and Referencing) shapefile from the US Census Bureau. We will filter to
use the Phoenix subdivision shapefile. 

## Purpose

The purpose of this analysis is to visually highlight areas within Phoenix have since lost BII values of greater than or equal to 0.75. 

As urbanization increasingly becomes prevalent in different areas, land cover changes then affects biodiversity of a given area. According to the USDA (US Department of Agriculture), "urban expansion may
lead to fragmentation, potentially resulting in genetic or demographic isolation of native species "(Ricketts 2001). It can be argued that urbanization can affect biodiversity and ecosystems in direct and indirect ways. Direct impacts of urbanization range anywhere from habitat loss to modified soils due to land transformation. Biodiversity is an important focal point of research because it contributes to priceless ecosystem services that cannot be replaced or replenished once depleted. It is important to take note at the loss in order to prevent further loss from occurring. 

According to Microsoft Planetary Computer (MPC), biodiversity intactness is a combination of two measurements : 
- the quantity of individuals
- Compositional Similarity (how similar the composition of species is to an intact baseline)

### Importing necessary libraries

In [1]:
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os
import contextily as ctx 
from shapely.geometry import Polygon
from shapely.geometry import box

# used to access STAC catalogs
from pystac_client import Client

import planetary_computer

#---- other libraries for nice outputs
from IPython.display import Image